In [20]:
from copy import deepcopy
import urllib.request, json, unidecode
import praw
import pandas as pd

import json
import nltk
from nltk.corpus import treebank
import copy
import re
import urllib
from bs4 import BeautifulSoup
import requests
import webbrowser
from nltk.corpus import ieer
import math



limit = 500

def addInUsa(text):
    print("addInUSA called :",text)
    text =text+",1\n"
    with open("myfile.csv","a") as file1:
        file1.write(text)
        file1.close()

def addInOthers(text):
    text = text+",0\n"
    with open("myfile.csv","a") as file2:
        file2.write(text)
        file2.close()


def isClean(title):
    title=str(title)
    if title.find("x") >= 0:
        k= title.find("x")
        if (k-1) >=0 and (not title[k-1].isalpha()):
            return False
        elif (k+1) < len(title) and ( not title[k+1].isalpha()):
            return False
    elif title.find("X") >= 0:
        k= title.find("X")
        if (k-1) >=0 and (not title[k-1].isalpha()):
            return False
        elif (k+1) < len(title) and (not title[k+1].isalpha()):
            return False

    elif title.find("×") >= 0:
        k= title.find("×")
        if (k-1) >=0 and (not title[k-1].isalpha()):
            return False
        elif (k+1) < len(title) and (not title[k+1].isalpha()):
            return False

    elif title.find("*") >= 0:
            k = title.find("*")
            if (k - 1) >= 0 and (not title[k - 1].isalpha()):
                return False
            elif (k + 1) < len(title) and (not title[k + 1].isalpha()):
                return False
    return True

def properNoun(lexical):
    noun =""

    for i in range(len(lexical)):
        if lexical[i][1]=="NNP" and str(lexical[i][0]).isalpha():
            noun+=lexical[i][0]+" "
        elif (i+1) < len(lexical) and str(lexical[i][0]).isalpha() and lexical[i-1][1]=="NNP" and lexical[i+1][1]=="NNP":
            noun += lexical[i][0] + " "
    if noun=="":
        for i in range(len(lexical)):
            if (lexical[i][1] == "NN" or lexical[i][1] == "NNS") and str(lexical[i][0]).isalpha() and str(lexical[i][0])[0].isupper():
                noun += lexical[i][0] + " "
            elif ((i + 1) < len(lexical) and (i-1) >=0) and lexical[i][1]!="IN" and str(lexical[i][0]).isalpha() and ((lexical[i-1][1] == "NN" and lexical[i + 1][1] == "NN") or (lexical[i-1][1] == "NNS" and lexical[i + 1][1] == "NNS")):
                noun += lexical[i][0] + " "

    return noun.strip()

def locateFormed(location):
    list = ""
    for triplet in location:
        list += triplet[0][0]+" "
    return list.strip()
def cleanTitle(title, step):
    if step==1:
        title = str(title)
        title = title.strip()

        motif = re.compile(r"\d{1,}")
        list = motif.findall(title)
        if len(list)>0:
            for out in list:
                title = title.replace(out,"")
        indice = title.find("IG")
        if indice >= 0:
            if (indice - 1) >= 0  and not title[indice - 1].isalpha():
                title = title.replace("IG", "")
        indice = title.find("ig")
        if indice >= 0:
            if (indice - 1) >= 0 and not title[indice - 1].isalpha() :
                title = title.replace("ig", "")

        motif = re.compile(r"[a-zA-Z]+[0-9]+")
        list = motif.findall(title)
        if len(list) > 0:
            for out in list:
                title = title.replace(out, "")

        title = title.replace("😉", "")
        title = title.replace("️","")
        title = title.replace("❤", "")
        title=title.replace("[OC]","")
        title = title.replace("{OC}", "")
        title = title.replace("{oc}", "")
        title = title.replace(",OC ", "")
        title = title.replace("OC, ", "")
        title = title.replace("(OC)", "")
        title = title.replace("(oc), ", "")
        title = title.replace("oc,", "")
        title = title.replace(",oc ", "")
        title = title.replace(":", "")
        title = title.replace("[oc]", "")
        title = title.replace("(", "")
        title = title.replace(")", "")
        title = title.replace("[", "")
        title = title.replace("]", "")
        title=title.replace(" OC ","")
        title = title.replace(" oc ", "")
        title = title.replace(",","") #on enlève les virgules pour csv
        title = title.replace(".","") #on enlève les points pour le csv
        indice = title.find("OC")
        if indice >= 0 and (indice+2)== len(title):
            title = title.replace("OC", "")
        indice = title.find("oc")
        if indice >= 0 and (indice + 2) == len(title):
            title = title.replace("oc", "")




        if title[:3]=="OC " or title[:3]=="oc ":
            title = title[3:]
        title = title.strip()
        if (title.find("@") >= 0):
            indice = title.find("@")
            k = indice + 1
            print(" avant",title)
            while ( k < len(title) and title[k]!=" " ):
                k = k + 1
            title = title[:indice] + title[k:]
            title = title.strip()


        title = title.strip()

        print("After clean :",title)
        return title
    if step==2:

        indice = title.find("x")
        if indice < 0:
            indice = title.find("X")
            if indice < 0:
                indice = title.find("×")
                if indice < 0:
                    indice = title.find("*")

        if indice >=0 and ((indice-1)>=0 and not title[indice-1].isalpha()) and ((indice+1)<len(title) and not title[indice+1].isalpha()):
            chain1 = str(title[:indice])
            chain2 = str(title[indice+1:])
            if len(chain1) > 0:
                while len(chain1)>0 and not chain1[-1].isalpha():
                    chain1 = chain1[:-1]
                while len(chain1)>0 and not chain1[0].isalpha():
                    chain1 = chain1[1:]
            else:
                chain1 = ""

            if len(chain2) > 1:
                while len(chain2)>0 and not chain2[0].isalpha():
                    chain2 = chain2[1:]
                while len(chain2)>0 and not chain2[-1].isalpha():
                    chain2 = chain2[:-1]
            else:
                chain2 = ""

            title = chain1+" "+chain2
            while len(title)>0 and not title[-1].isalpha():
                title = title[:-1]
            title = title.strip()
            print("After clean :", title)
            return title
        elif (indice + 1) == len(title) and (indice-1)>= 0 and  not title[indice - 1].isalpha():
            title = title[:-2]
        elif (indice == 0) and (indice + 1 ) < len(title) and  not title[indice + 1].isalpha():
            title = title[1:]
        print("After clean :", title)

    return title

def distanceBetweenCordinates(coord1, coord2):
    coord1[0] = math.radians(coord1[0])
    coord1[1] = math.radians(coord1[1])
    coord2[0] = math.radians(coord2[0])
    coord2[1] = math.radians(coord2[1])
    SPHERE_TERRE = 6378137
    DIST_LON = (coord2[0] - coord1[0]) / 2
    DIST_LAT = (coord2[1] - coord1[1]) / 2
    ETAPE1 = (math.sin(DIST_LAT)**2) + math.cos(coord1[1]) * math.cos(coord2[1]) * (math.sin(DIST_LON)**2)
    ETAPE2 = 2 * math.atan2(math.sqrt(ETAPE1), math.sqrt(1 - ETAPE1))
    resultat = (SPHERE_TERRE * ETAPE2) / 1000
    return resultat

def existNameSocialNetwork(title):

    title = str(title)
    indice = title.find("Insta")
    if indice < 0:
        indice = title.find("Instagram")
        if indice < 0:
            indice = title.find("instagram")
            if indice < 0:
                indice = title.find("instaGram")
                if indice < 0:
                    indice = title.find("insta")
                    if indice < 0:
                        indice = title.find("Facebook")
                        if indice < 0:
                            indice = title.find("facebook")
                            if indice < 0:
                                indice = title.find("FaceBook")
                                if indice < 0:
                                    indice = title.find("Fb")
                                    if indice < 0:
                                        indice = title.find("fb")
                                        if indice < 0:
                                            indice = title.find("FB")


    if indice >= 0:
        indice = indice + 1
        a = title[:indice-1]
        while indice < len(title) and title[indice] != " ":
            indice = indice + 1
        title = a+title[indice:]
        return existNameSocialNetwork(title)
    return title
def geoNamesSearch(lieu):
    lieu = lieu.replace(" ", "+")       #remplace les espaces par des +
    lieu = lieu.replace("'", "+")       #remplace les apostrophes par des plus
    lieu = unidecode.unidecode(lieu)    #pour retirer les accents !attention tester les cédilles

    api = "http://api.geonames.org/searchJSON?q="+lieu+"&maxRows=1&username=projet_TER_reddit"
    #print(api)

    with urllib.request.urlopen(api) as url:
        data = json.loads(url.read().decode())
    #print("Taille:", len(data))
    if data['totalResultsCount'] == 0:
        return -1

    long = data['geonames'][0]['lng']
    lat  = data['geonames'][0]['lat']
    contry = ""
    code = ""
    name = ""
    if "countryName" in data['geonames'][0] and "countryCode" in data['geonames'][0]:
        contry = data['geonames'][0]["countryName"]
        code = data['geonames'][0]["countryCode"]
        name = data['geonames'][0]["name"]
    else:
        contry = data['geonames'][0]["fcodeName"]
        code = data['geonames'][0]["fcode"]
        name = data['geonames'][0]["name"]


    list=[]

    list.append(lieu)
    list.append(long)
    list.append(lat)
    list.append(contry)
    list.append(code)
    list.append(name)
    return list



def findLocation(liste):
    list = []
    for triple in liste:
        if len(triple[0][1]) >2 and  (triple[1]=="GSP" or triple[1]=="GPE" or triple[1]=="ORGANIZATION" or triple[1]=="PERSON"):
            list.append(triple)
    return list

def equalsTextList(list1, list2):
    if len(list1)!=len(list2):
        return  False
    for i in range(len(list2)):
        if list1[i]!=list2[i][0][0]:
            return False
    return True

def middleCheck(list1, list2):

    #check ineteressant
    composed = locateFormed(list2)
    i = len(list1)-1
    while i > 0 :
        composed =list1[i][0][0]+" "+composed
        find = geoNamesSearch(composed)
        if find != -1:
            return find
        i= i-1

        # check du list2
    find = geoNamesSearch(locateFormed(list2))
    if find != -1:
        return find

        # check pour list1
    if len(list1) >= 2:
            find = geoNamesSearch(locateFormed(list1))
            if find != -1:
                return find

    #check dichotomique
    i = 0
    composed = ""
    for word in reversed(list1):
        composed = word[0][0] +" "+ composed+(" " if len(composed)>0 else "") + list2[i][0][0]
        i = i+1
        find = geoNamesSearch(composed)
        if find != -1:
            return  find

    while i < len(list2):
        composed  += " " + list2[i][0][0]
        i = i+1
        find = geoNamesSearch(locateFormed(composed))
        if find != -1:
            return find
    composed = ""
    for word in reversed(list2):
        composed+=word[0][0]
        find = geoNamesSearch(composed)
        composed +=" "
        if find != -1:
            return find
    return -1
def fusion():
    tab = []
    p = pd.read_json("reddit.json")
    taille = p.shape[1] + 1
    for i in p:
        tab.append([p[i].title, p[i].afterClean, p[i].subreddit, p[i].url, p[i].body, p[i].location, p[i].longitude,
                    p[i].latitude, p[i].contryName, p[i].contryCode, p[i].name])

    f = pd.read_json("fusion.json")
    for i in f:
        tab.append([f[i].title, f[i].afterClean, f[i].subreddit, f[i].url, f[i].body, f[i].location, f[i].longitude,
                    f[i].latitude, f[i].contryName,
                    f[i].contryCode, f[i].name])

    tab = pd.DataFrame(tab,
                       columns=['title', 'afterClean', 'subreddit', 'url', 'body', 'location', 'longitude', 'latitude',
                                'contryName', 'contryCode', 'name'])

    tab.to_json("reddit.json", orient='index')


def collectionFromReddit():
    ok = 0

    reddit = praw.Reddit ( client_id = 'tMRO7I9OYVnG7A' , client_secret = 'Ghudpyy79xN1dUv9-lIdRaiTswE' , user_agent = 'dassScraping' )
    posts = []
    ml_subreddit = reddit.subreddit('EarthPorn')
    for post in ml_subreddit.hot(limit=limit):
        print("Original title :",post.title)
        step =1
        afterClean = existNameSocialNetwork(deepcopy(post.title))
        afterClean=cleanTitle(afterClean,step)
        print("It's cleaned ?",isClean(afterClean))
        while not isClean(afterClean) and step <=2:
            step = step + 1
            afterClean = cleanTitle(afterClean, step)
            print("It's cleaned ?", isClean(afterClean))
        lexical = nltk.word_tokenize(afterClean)
        lexical = nltk.pos_tag(lexical)
        print("lexical:",lexical)
        proper = properNoun(lexical)
        print("proper noun:", proper)
        if len(proper) > 0:
            resultat = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(afterClean)))
            print("Test :",resultat.pos())
            loc = findLocation(resultat.pos())
            print("Location: ",loc)
            coordinate = geoNamesSearch(proper)
            print("geoNames:", coordinate)
            ok = ok + 1
            check = True
            if coordinate == -1 :
                ok = ok - 1
                if not equalsTextList(proper.split(),loc) and len(loc)>0:
                    coordinate = geoNamesSearch(locateFormed(loc))
                    if coordinate != -1:
                        ok = ok + 1
                        check = False
                    print("geoNames:",coordinate)

                    if check:
                        debut = loc[:len(loc) // 2]
                        fin = loc[len(loc) // 2:]

                        coordinate = middleCheck(debut,fin)
                        if coordinate != -1:
                            ok = ok + 1
                        print("geonames:",coordinate)
                elif len(loc)>1:
                    debut = loc[:len(loc) // 2]
                    fin = loc[len(loc) // 2:]

                    coordinate = middleCheck(debut, fin)
                    if coordinate != -1:
                        ok = ok + 1
                    print("geonames:", coordinate)

        print("-------------------------------------")
        if coordinate!=-1 and len(coordinate)== 6:
            posts.append([str(post.title), afterClean, str(post.subreddit), ""+str(post.url), str(post.selftext).strip(' \\'),coordinate[0],float(coordinate[1]),float(coordinate[2]),coordinate[3],str(coordinate[4]), coordinate[5]])
        
        if coordinate != -1 and coordinate[4] == "US":
            addInUsa(afterClean)
            #print("TEST COORDONNEE")
            
        else:
            addInOthers(afterClean)

    posts = pd.DataFrame(posts,columns=['title', 'afterClean', 'subreddit', 'url', 'body','location','longitude','latitude','contryName','contryCode','name'])
    print(ok,"Trouvés sur",limit)
    posts.to_json("fusion.json", orient='index')
    fusion()

collectionFromReddit()


#lexical = nltk.word_tokenize("National Parc of Jacques-Cartier")
#lexical = nltk.pos_tag(lexical)
#print("lexical:",lexical)


Original title : Estes Park, Colorado [3024 x 4032] OC
After clean : Estes Park Colorado  x
It's cleaned ? False
After clean : Estes Park Colorado 
It's cleaned ? True
lexical: [('Estes', 'NNS'), ('Park', 'NNP'), ('Colorado', 'NNP')]
proper noun: Estes Park Colorado
Test : [(('Estes', 'NNS'), 'PERSON'), (('Park', 'NNP'), 'ORGANIZATION'), (('Colorado', 'NNP'), 'ORGANIZATION')]
Location:  [(('Estes', 'NNS'), 'PERSON'), (('Park', 'NNP'), 'ORGANIZATION'), (('Colorado', 'NNP'), 'ORGANIZATION')]
geoNames: ['Estes+Park+Colorado', '-105.52167', '40.37721', 'United States', 'US', 'Estes Park']
-------------------------------------
addInUSA called : Estes Park Colorado 
Original title : The most beautiful mountain range in the Dolomites [OC][1805x1920] IG: @holysh0t
 avant The most beautiful mountain range in the Dolomites x  @holysht
After clean : The most beautiful mountain range in the Dolomites x
It's cleaned ? False
After clean : The most beautiful mountain range in the Dolomites
It's clean

geonames: ['Oswald+West+Park+Oregon', '-123.95986', '45.76983', 'United States', 'US', 'Oswald West State Park']
-------------------------------------
addInUSA called : Hidden cove at Oswald West Park Oregon
Original title : The Ghost Mountains, Kamchatka (Photo: Isabella Tabacchi) [OS] [1080x810]
After clean : The Ghost Mountains Kamchatka Photo Isabella Tabacchi OS x
It's cleaned ? False
After clean : The Ghost Mountains Kamchatka Photo Isabella Tabacchi OS
It's cleaned ? True
lexical: [('The', 'DT'), ('Ghost', 'NNP'), ('Mountains', 'NNP'), ('Kamchatka', 'NNP'), ('Photo', 'NNP'), ('Isabella', 'NNP'), ('Tabacchi', 'NNP'), ('OS', 'NNP')]
proper noun: The Ghost Mountains Kamchatka Photo Isabella Tabacchi OS
Test : [(('The', 'DT'), 'S'), (('Ghost', 'NNP'), 'ORGANIZATION'), (('Mountains', 'NNP'), 'ORGANIZATION'), (('Kamchatka', 'NNP'), 'ORGANIZATION'), (('Photo', 'NNP'), 'ORGANIZATION'), (('Isabella', 'NNP'), 'ORGANIZATION'), (('Tabacchi', 'NNP'), 'ORGANIZATION'), (('OS', 'NNP'), 'S')]
Lo

geoNames: -1
geonames: ['National+Park+Oregon', '-122.13282', '42.94109', 'United States', 'US', 'Crater Lake National Park']
-------------------------------------
addInUSA called : Sunset at Ecola National Park Oregon
Original title : The Grand Canyon, Arizona [4032x3024] (OC)
After clean : The Grand Canyon Arizona x
It's cleaned ? False
After clean : The Grand Canyon Arizona
It's cleaned ? True
lexical: [('The', 'DT'), ('Grand', 'NNP'), ('Canyon', 'NNP'), ('Arizona', 'NNP')]
proper noun: The Grand Canyon Arizona
Test : [(('The', 'DT'), 'S'), (('Grand', 'NNP'), 'FACILITY'), (('Canyon', 'NNP'), 'FACILITY'), (('Arizona', 'NNP'), 'FACILITY')]
Location:  []
geoNames: ['The+Grand+Canyon+Arizona', '-113.99607', '36.11276', 'United States', 'US', 'Grand Canyon']
-------------------------------------
addInUSA called : The Grand Canyon Arizona
Original title : Looking forward to summer again. Gimmelwald, Switzerland [3790x5305] [OC]
After clean : Looking forward to summer again Gimmelwald Swit

geoNames: -1
geoNames: -1
geonames: ['Sunrise+Arizona', '-109.74539', '35.60362', 'United States', 'US', 'Sunrise Springs']
-------------------------------------
addInUSA called : Spring Flower at Sunrise in Arizona
Original title : A soft sunset at the UK's Pulpit Rock in Dorset [OC] [4088x5110] IG: @pete_ell
 avant A soft sunset at the UK's Pulpit Rock in Dorset  x  @pete_ell
After clean : A soft sunset at the UK's Pulpit Rock in Dorset  x
It's cleaned ? False
After clean : A soft sunset at the UK's Pulpit Rock in Dorset 
It's cleaned ? True
lexical: [('A', 'DT'), ('soft', 'JJ'), ('sunset', 'NN'), ('at', 'IN'), ('the', 'DT'), ('UK', 'NNP'), ("'s", 'POS'), ('Pulpit', 'NNP'), ('Rock', 'NNP'), ('in', 'IN'), ('Dorset', 'NNP')]
proper noun: UK Pulpit Rock in Dorset
Test : [(('A', 'DT'), 'S'), (('soft', 'JJ'), 'S'), (('sunset', 'NN'), 'S'), (('at', 'IN'), 'S'), (('the', 'DT'), 'S'), (('UK', 'NNP'), 'ORGANIZATION'), (("'s", 'POS'), 'S'), (('Pulpit', 'NNP'), 'PERSON'), (('Rock', 'NNP'), 'PER

geoNames: ['Rio+Negro+Amazon+Forest+Amazon+Forest', '-68.5', '1.11667', 'Brazil', 'BR', 'Reserva Florestal do Rio Negro']
-------------------------------------
Original title : Sunset in Tombstone Territorial Park, Yukon, Canada [6192 × 4128] [OC]
After clean : Sunset in Tombstone Territorial Park Yukon Canada  ×
It's cleaned ? False
After clean : Sunset in Tombstone Territorial Park Yukon Canada 
It's cleaned ? True
lexical: [('Sunset', 'NNP'), ('in', 'IN'), ('Tombstone', 'NNP'), ('Territorial', 'NNP'), ('Park', 'NNP'), ('Yukon', 'NNP'), ('Canada', 'NNP')]
proper noun: Sunset in Tombstone Territorial Park Yukon Canada
Test : [(('Sunset', 'NNP'), 'GPE'), (('in', 'IN'), 'S'), (('Tombstone', 'NNP'), 'GPE'), (('Territorial', 'NNP'), 'S'), (('Park', 'NNP'), 'PERSON'), (('Yukon', 'NNP'), 'PERSON'), (('Canada', 'NNP'), 'PERSON')]
Location:  [(('Sunset', 'NNP'), 'GPE'), (('Tombstone', 'NNP'), 'GPE'), (('Park', 'NNP'), 'PERSON'), (('Yukon', 'NNP'), 'PERSON'), (('Canada', 'NNP'), 'PERSON')]
geo

...

geoNames: -1
geoNames: ['Koke', '39.73333', '-69.21389', 'Antarctica', 'AQ', 'Koke Strand']
-------------------------------------
Original title : Aekingerzand in the summer. [OC] [5571 x 3714]
After clean : Aekingerzand in the summer   x
It's cleaned ? False
After clean : Aekingerzand in the summer  
It's cleaned ? True
lexical: [('Aekingerzand', 'NNP'), ('in', 'IN'), ('the', 'DT'), ('summer', 'NN')]
proper noun: Aekingerzand
Test : [(('Aekingerzand', 'NNP'), 'GPE'), (('in', 'IN'), 'S'), (('the', 'DT'), 'S'), (('summer', 'NN'), 'S')]
Location:  [(('Aekingerzand', 'NNP'), 'GPE')]
geoNames: -1
-------------------------------------
Original title : Lake Tekapo, New Zealand (4128x2322) [OC]
After clean : Lake Tekapo New Zealand x
It's cleaned ? False
After clean : Lake Tekapo New Zealand
It's cleaned ? True
lexical: [('Lake', 'NNP'), ('Tekapo', 'NNP'), ('New', 'NNP'), ('Zealand', 'NNP')]
proper noun: Lake Tekapo New Zealand
Test : [(('Lake', 'NNP'), 'PERSON'), (('Tekapo', 'NNP'), 'PERSON'

geoNames: ['CGN+Germany', '6.95', '50.93333', 'Germany', 'DE', 'Cologne']
-------------------------------------
Original title : Better bays ahead - Central Coast, CA [OC] [6000x4000]
After clean : Better bays ahead - Central Coast CA  x
It's cleaned ? False
After clean : Better bays ahead - Central Coast CA 
It's cleaned ? True
lexical: [('Better', 'NNP'), ('bays', 'VBZ'), ('ahead', 'RB'), ('-', ':'), ('Central', 'NNP'), ('Coast', 'NNP'), ('CA', 'NNP')]
proper noun: Better Central Coast CA
Test : [(('Better', 'NNP'), 'PERSON'), (('bays', 'VBZ'), 'S'), (('ahead', 'RB'), 'S'), (('-', ':'), 'S'), (('Central', 'NNP'), 'ORGANIZATION'), (('Coast', 'NNP'), 'ORGANIZATION'), (('CA', 'NNP'), 'S')]
Location:  [(('Better', 'NNP'), 'PERSON'), (('Central', 'NNP'), 'ORGANIZATION'), (('Coast', 'NNP'), 'ORGANIZATION')]
geoNames: -1
geoNames: -1
geonames: ['Central+Coast', '107.58478', '16.47129', 'Vietnam', 'VN', 'North Central Coast']
-------------------------------------
Original title : Clear day i

geoNames: -1
geoNames: -1
geonames: ['Moab', '-109.54984', '38.57332', 'United States', 'US', 'Moab']
-------------------------------------
addInUSA called : Arches Moab UT
Original title : South Africa, 2016 (4032x3024) OC
After clean : South Africa  x
It's cleaned ? False
After clean : South Africa 
It's cleaned ? True
lexical: [('South', 'NNP'), ('Africa', 'NNP')]
proper noun: South Africa
Test : [(('South', 'NNP'), 'GPE'), (('Africa', 'NNP'), 'ORGANIZATION')]
Location:  [(('South', 'NNP'), 'GPE'), (('Africa', 'NNP'), 'ORGANIZATION')]
geoNames: ['South+Africa', '24', '-29', 'South Africa', 'ZA', 'South Africa']
-------------------------------------
Original title : Starina water reservoir, Eastern Slovakia [OC] [3000x2003]
After clean : Starina water reservoir Eastern Slovakia  x
It's cleaned ? False
After clean : Starina water reservoir Eastern Slovakia 
It's cleaned ? True
lexical: [('Starina', 'NNP'), ('water', 'NN'), ('reservoir', 'NN'), ('Eastern', 'NNP'), ('Slovakia', 'NNP')]


geoNames: ['Dyrholaey+Iceland', '-19.12777', '63.39871', 'Iceland', 'IS', 'Portland Point']
-------------------------------------
Original title : My Amateur Attempt at a Waterfall Long Exposure | Upper Lewis Falls, Washington State [3265x4898][OC]
After clean : My Amateur Attempt at a Waterfall Long Exposure | Upper Lewis Falls Washington State x
It's cleaned ? True
lexical: [('My', 'PRP$'), ('Amateur', 'JJ'), ('Attempt', 'NNP'), ('at', 'IN'), ('a', 'DT'), ('Waterfall', 'NNP'), ('Long', 'NNP'), ('Exposure', 'NNP'), ('|', 'NNP'), ('Upper', 'NNP'), ('Lewis', 'NNP'), ('Falls', 'NNP'), ('Washington', 'NNP'), ('State', 'NNP'), ('x', 'NN')]
proper noun: Attempt Waterfall Long Exposure Upper Lewis Falls Washington State
Test : [(('My', 'PRP$'), 'S'), (('Amateur', 'JJ'), 'ORGANIZATION'), (('Attempt', 'NNP'), 'ORGANIZATION'), (('at', 'IN'), 'S'), (('a', 'DT'), 'S'), (('Waterfall', 'NNP'), 'ORGANIZATION'), (('Long', 'NNP'), 'ORGANIZATION'), (('Exposure', 'NNP'), 'ORGANIZATION'), (('|', 'NNP'), 

KeyError: 'totalResultsCount'